In [ ]:
!pip install pyTelegramBotAPI nltk gtts pydub python-docx pdfminer.six googletrans==4.0.0-rc1
!pip install python-docx
!pip install docx2txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.2/232.2 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 35.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 59.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 7.3 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17396 sha256=3d95f7017e0b9194f1036f29d48284b1411a631f8f1e973f09ff9f4

In [ ]:
!pip install rouge-score
!pip install PyPDF2

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=c9f017af7aa06103b8a0fdcf6adf01d337a634faf05a613e5a5f9df23333c211
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.4 MB/s eta 0:00:00


In [ ]:
import telebot
import nltk
from gtts import gTTS
from PIL import Image, ImageDraw, ImageFont
import cv2
import numpy as np
import os
from rouge_score import rouge_scorer
from googletrans import Translator
import requests
from bs4 import BeautifulSoup
import time
from PyPDF2 import PdfReader
import docx

# Initialize the Telegram bot
bot = telebot.TeleBot("6414110367:AAEdLJLl8iEFMR7KUgZsDaC8aLzySNtZsH0")

# Download NLTK resources
nltk.download('punkt')

# Initialize global variables for tracking processing time, user feedback, and MOS (Mean Opinion Score)
processing_times = {}
user_feedback = {}
mos_scores = {}

def summarize_text(input_text):
    start_time = time.time()
    sentences = nltk.sent_tokenize(input_text)
    summary = ' '.join(sentences[:3])
    end_time = time.time()
    processing_time = end_time - start_time
    return summary, processing_time

def text_to_speech(text, language_code, audio_path):
    start_time = time.time()
    tts = gTTS(text=text, lang=language_code, slow=False)
    tts.save(audio_path)
    end_time = time.time()
    processing_time = end_time - start_time
    return processing_time

def text_to_video(summarized_text, language_code, video_filename):
    start_time = time.time()
    # Set video dimensions and frame rate
    width, height = 1280, 720
    fps = 10  # Decreased frame rate for slower movement
    frame_duration = 3  # Duration each frame is displayed (in seconds)

    # Set font properties based on language
    if language_code == 'en':
        font_path = "/content/sample_data/OpenSans-Regular.ttf"
    elif language_code == 'ta':
        font_path = "/content/sample_data/SaiIndira.ttf"
    elif language_code == 'hi':
        font_path = "/content/sample_data/Kruti_Dev_040_Wide.ttf"
    # Add font paths for other languages as needed

    font = ImageFont.truetype(font_path, 50)
    text_color = (255, 255, 255)  # White color

    # Split summarized text into lines to fit within video width
    lines = []
    line = ''
    for word in summarized_text.split():
        if font.getsize(line + word)[0] <= width:
            line += f"{word} "
        else:
            lines.append(line)
            line = f"{word} "
    lines.append(line)

    # Write text to frames
    frame_texts = []
    for line in lines:
        frame = np.zeros((height, width, 3), dtype=np.uint8)
        pil_frame = Image.fromarray(frame)
        draw = ImageDraw.Draw(pil_frame)
        draw.text((10, 10), line, font=font, fill=text_color)
        frame_texts.append(np.array(pil_frame))

    # Initialize video writer
    out = cv2.VideoWriter(video_filename, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    # Write frames to video
    for frame_text in frame_texts:
        for _ in range(int(frame_duration * fps)):
            out.write(frame_text)
    out.release()
    end_time = time.time()
    processing_time = end_time - start_time
    return processing_time

def extract_text_from_url(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    paragraphs = soup.find_all('p')
    text = '\n'.join([p.get_text() for p in paragraphs])
    return text

def extract_text_from_pdf(file_path):
    with open(file_path, 'rb') as file:
        pdf_reader = PdfReader(file)
        text = ''
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text

def extract_text_from_docx(file_path):
    doc = docx.Document(file_path)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return '\n'.join(full_text)

@bot.message_handler(commands=['start', 'help'])
def send_welcome(message):
    bot.reply_to(message, "Welcome to the Text Summarization Chatbot! Please send me a text, a URL, a PDF, or a Word file (max 3000 words) to summarize.")

@bot.message_handler(content_types=['text', 'document'])
def summarize_and_convert(message):
    if message.content_type == 'text':
        input_text = message.text
        if input_text.startswith('http://') or input_text.startswith('https://'):
            input_text = extract_text_from_url(input_text)
    else:
        file_info = bot.get_file(message.document.file_id)
        downloaded_file = bot.download_file(file_info.file_path)
        if message.document.mime_type == 'application/pdf':
            with open('document.pdf', 'wb') as file:
                file.write(downloaded_file)
            input_text = extract_text_from_pdf('document.pdf')
        elif message.document.mime_type == 'application/vnd.openxmlformats-officedocument.wordprocessingml.document':
            with open('document.docx', 'wb') as file:
                file.write(downloaded_file)
            input_text = extract_text_from_docx('document.docx')
        else:
            bot.reply_to(message, "Unsupported file format. Please provide a PDF or Word document.")
            return

    if len(input_text.split()) > 3000:
        bot.reply_to(message, "The input text is too long. Please provide a text, URL, PDF, or Word file with a maximum of 3000 words.")
        return

    summarized_text, summarization_time = summarize_text(input_text)

    # Calculate ROUGE score
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    scores = scorer.score(input_text, summarized_text)
    rouge_score = scores['rougeL'].fmeasure

    # Store processing time
    processing_times[message.chat.id] = {
        'summarization_time': summarization_time,
        'audio_processing_time': None,
        'video_processing_time': None
    }

    # Convert summarized text to audio
    language_keyboard = telebot.types.ReplyKeyboardMarkup(one_time_keyboard=True, resize_keyboard=True)
    language_keyboard.add("en", "ta", "hi", "ml", "te", "kn", "gu")  # Add more languages as needed
    language_prompt = bot.send_message(message.chat.id, "Please choose a language for audio conversion:", reply_markup=language_keyboard)
    bot.register_next_step_handler(language_prompt, process_language_selection, summarized_text, input_text, rouge_score)

def process_language_selection(message, summarized_text, input_text, rouge_score):
    chosen_language = message.text.lower()

    if chosen_language in ["en", "ta", "hi", "ml", "te", "kn", "gu"]:
        language_code = chosen_language[:2]  # Extract language code (e.g., 'en', 'fr', 'es')

        # Translate the summarized text to the selected language
        translator = Translator()
        translated_text = translator.translate(summarized_text, dest=language_code).text

        # Convert translated text to audio
        audio_path = "summarized_audio.mp3"
        audio_processing_time = text_to_speech(translated_text, language_code, audio_path)
        processing_times[message.chat.id]['audio_processing_time'] = audio_processing_time
        bot.send_audio(message.chat.id, open(audio_path, 'rb'))

        # Ask if the user wants to convert audio to video
        video_conversion_keyboard = telebot.types.ReplyKeyboardMarkup(one_time_keyboard=True, resize_keyboard=True)
        video_conversion_keyboard.add("Yes", "No")
        video_conversion_prompt = bot.send_message(message.chat.id, "Do you want to convert the summarized text to video?", reply_markup=video_conversion_keyboard)
        bot.register_next_step_handler(video_conversion_prompt, process_video_conversion_selection, summarized_text, language_code, rouge_score)

    else:
        bot.reply_to(message, "Invalid language selection. Please choose a valid language.")

def process_video_conversion_selection(message, summarized_text, language_code, rouge_score):
    user_response = message.text.lower()
    if user_response == "yes":
        # Convert summarized text to video
        video_filename = "summarized_video.mp4"
        video_processing_time = text_to_video(summarized_text, language_code, video_filename)
        processing_times[message.chat.id]['video_processing_time'] = video_processing_time

        # Calculate MOS (Mean Opinion Score)
        mos_score = calculate_mos(processing_times[message.chat.id]['summarization_time'])
        mos_scores[message.chat.id] = mos_score

        # Merge video with audio
        audio_path = "summarized_audio.mp3"
        audio_processing_time = processing_times[message.chat.id]['audio_processing_time']
        merge_audio_to_video(audio_path, video_filename, message.chat.id)

        # Send final summary with ROUGE score, MOS, processing time, and user feedback
        processing_time = sum(processing_times[message.chat.id].values())
        final_summary = f"ROUGE Score: {rouge_score}\nMOS: {mos_score}\nProcessing Time: {processing_time} seconds\n\nPlease rate the quality of the summary and the audio-video synchronization on a scale of 1 to 5."
        feedback_prompt = bot.send_message(message.chat.id, final_summary)
        bot.register_next_step_handler(feedback_prompt, process_user_feedback, message.chat.id)

    elif user_response == "no":
        bot.reply_to(message, "Okay, not converting summarized text to video. Type /start to summarize again.")
    else:
        bot.reply_to(message, "Invalid response. Please choose either 'Yes' or 'No'.")

def calculate_mos(processing_time):
    # Define thresholds for processing time and corresponding scores
    thresholds = [10, 20, 30]  # Define thresholds in seconds
    scores = [5, 4, 3, 2, 1]    # Corresponding scores for each threshold

    # Iterate through thresholds to find the corresponding score
    for threshold, score in zip(thresholds, scores):
        if processing_time <= threshold:
            return score

    # If processing time exceeds all thresholds, return the lowest score
    return scores[-1]


def merge_audio_to_video(audio_path, video_filename, chat_id):
    video_with_audio_filename = "summarized_video_with_audio.mp4"
    # Use ffmpeg to merge audio with video
    os.system(f"ffmpeg -i {video_filename} -i {audio_path} -c:v copy -c:a aac -strict experimental {video_with_audio_filename}")
    # Send the video with audio
    bot.send_video(chat_id, open(video_with_audio_filename, 'rb'))

def process_user_feedback(message, chat_id):
    user_response = message.text
    if user_response.isdigit() and 1 <= int(user_response) <= 5:
        user_feedback[chat_id] = int(user_response)
        bot.reply_to(message, "Thank you for your feedback!")

        # Prompt the user for starting a new session or ending the current session
        session_prompt = bot.send_message(chat_id, "Do you want to start a new session or end the current session?\n\nIf Starting a new session, type /start to begin.\nIf Ending the current session, type /end.")
        bot.register_next_step_handler(session_prompt, process_session_decision, chat_id)
    else:
        bot.reply_to(message, "Invalid feedback. Please rate on a scale of 1 to 5.")

def process_session_decision(message, chat_id):
    decision = message.text.lower()
    if decision == "/start":
        bot.send_message(chat_id, "Starting a new session. Type /start to begin.")
    elif decision == "/end":
        bot.send_message(chat_id, "Ending the current session. Type /start to begin a new session.")
    else:
        bot.send_message(chat_id, "Invalid decision. Please type /start or /end.")

# Start the bot
bot.polling()
